<a href="https://colab.research.google.com/github/kmrtanmay/pytorch-tutorial/blob/master/pytorch_tutorial/basic_feedforward_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [360]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [0]:
# Hyper-parameters 
input_size = 784
hidden_size = 500

num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001


In [0]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [0]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


In [366]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3455
Epoch [1/5], Step [200/600], Loss: 0.1602
Epoch [1/5], Step [300/600], Loss: 0.2737
Epoch [1/5], Step [400/600], Loss: 0.1001
Epoch [1/5], Step [500/600], Loss: 0.3010
Epoch [1/5], Step [600/600], Loss: 0.2479
Epoch [2/5], Step [100/600], Loss: 0.1190
Epoch [2/5], Step [200/600], Loss: 0.1569
Epoch [2/5], Step [300/600], Loss: 0.0881
Epoch [2/5], Step [400/600], Loss: 0.3368
Epoch [2/5], Step [500/600], Loss: 0.0386
Epoch [2/5], Step [600/600], Loss: 0.1265
Epoch [3/5], Step [100/600], Loss: 0.0901
Epoch [3/5], Step [200/600], Loss: 0.0518
Epoch [3/5], Step [300/600], Loss: 0.0779
Epoch [3/5], Step [400/600], Loss: 0.1387
Epoch [3/5], Step [500/600], Loss: 0.0246
Epoch [3/5], Step [600/600], Loss: 0.0517
Epoch [4/5], Step [100/600], Loss: 0.0477
Epoch [4/5], Step [200/600], Loss: 0.0605
Epoch [4/5], Step [300/600], Loss: 0.0216
Epoch [4/5], Step [400/600], Loss: 0.0602
Epoch [4/5], Step [500/600], Loss: 0.0387
Epoch [4/5], Step [600/600], Loss:

In [367]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency) 
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
        

Accuracy of the network on the 10000 test images: 97.87 %


hidden size ......  accuracy

    1             27.85
    2             60.21
    5             88.88
    20            94.46
    50            96.50
    200           97.55
    400           97.58
    450           97.82
    500           98.05
    550           97.89
    600           97.88
    1000          97.63
    1500          98.06
    2000          98.19
    5000          97.76

epochs..accuracy

### 4.......97.71
### 5.......97.89
### 6.......98.0
### 8.......97.95


```
learning rate    accuracy
0.0005           97.48
0.001            98.05
0.005            97.65
0.01             96.77
0.05             92.55
```





In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')